# Importando libs

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, Dropdown
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path
import re
import numpy as np
import json
from ipyfilechooser import FileChooser
from IPython.display import display
from tkinter import Tk, filedialog
from pathlib import Path

# Lendo o dataset

In [3]:
try:   
   df_processed = pd.read_csv('../data/prouni_2005_2019_processed.csv')
except Exception as e:
    print(f'Error: {e}')

#df_processed.head()

In [4]:
lista_colunas = df_processed.columns
lista_colunas

Index(['ANO_CONCESSAO', 'INSTITUICAO', 'TIPO', 'MODALIDADE_ENSINO', 'CURSO',
       'TURNO', 'SEXO', 'RACA', 'DEFICIENTE_FISICO', 'REGIAO', 'UF',
       'MUNICIPIO', 'IDADE'],
      dtype='object')

# Gráficos

### Funções para criar botão de salvar gráfico em png

In [5]:
def limpar_nome_arquivo(name: str, max_len: int = 160) -> str:
    name = (name or "grafico").strip()
    name = re.sub(r"\s+", " ", name)
    name = re.sub(r'[\\/:*?"<>|]+', "_", name)  # proibidos no Windows
    name = name.strip(" ._")
    if not name:
        name = 'grafico'
    return name[:max_len]
   
def salvar_plotly_png(fig, folder: str, default_title: str = 'grafico'):
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)

    title_text = getattr(getattr(getattr(fig, 'layout', None), 'title', None), 'text', None) or default_title
    base = _limpar_nome_arquivo(str(title_text))

    outpath = folder_path / f'{base.replace(' ', '_')}.png'
    i = 2
    while outpath.exists():
        outpath = folder_path / f'{base.replace(' ', '_')}_{i}.png'
        i += 1  

    fig.write_image(str(outpath).lower().replace(' ', '_'))
    return str(outpath)
    
def salvar_plotly_png(fig, folder: str, default_title: str = 'grafico'):
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)

    title_text = getattr(getattr(getattr(fig, 'layout', None), 'title', None), 'text', None) or default_title
    base = limpar_nome_arquivo(str(title_text))

    outpath = folder_path / f'{base}.png'
    i = 2
    while outpath.exists():
        outpath = folder_path / f'{base.replace(' ', '_')}_{i}.png'
        i += 1  

    fig.write_image(str(outpath).lower().replace(' ', '_'))
    return str(outpath)

def fazer_plotly_widget_com_salvar(
    plot_fn,
    controls: dict,
    folder: str,
    button_label: str = 'Salvar gráfico em PNG',
    default_title: str = 'grafico',
    run_once: bool = True,
):
    state = {'fig': None}

    out_plot = widgets.Output()
    msg = widgets.HTML('')

    btn = widgets.Button(description=button_label, icon='save', tooltip='Salva o gráfico atual em PNG')

    def _renderizar(change=None):
        
        with out_plot:
            out_plot.clear_output()
            kwargs = {k: w.value for k, w in controls.items()}
            fig = plot_fn(**kwargs)
            state['fig'] = fig
            try:
                fig.show()
            except AttributeError:
                print('Nenhum gráfico para mostrar')
            except Exception as e:
                print(f'Erro ao mostrar gráfico: {e}')

    def _on_salvar(_):
        fig = state['fig']
        if fig is None:
            msg.value = "<span style='color:#b00'>Ainda não existe gráfico para salvar.</span>"
            return
        try:
            path = salvar_plotly_png(fig, folder, default_title=str(default_title.replace(' ', '_')))
            msg.value = f"<span style='color:#060'>Salvo em: {path.replace(' ', '_')}</span>"
        except Exception as e:
            msg.value = f"<span style='color:#b00'>Erro ao salvar: {e}</span>"

    btn.on_click(_on_salvar)

    for w in controls.values():
        w.observe(_renderizar, names='value')

    ui = widgets.HBox([*controls.values(), btn, msg])

    if run_once:
        _renderizar()

    display(ui, out_plot)

    #return {'ui': ui, 'out': out_plot, 'button': btn, 'message': msg, 'state': state}

### Copiando e renomeando colunas

In [6]:
df_chars = df_processed.copy()

In [7]:
df_chars['IDADE'].describe()

count    2.685770e+06
mean     3.136613e+01
std      7.952443e+00
min      1.500000e+01
25%      2.500000e+01
50%      3.000000e+01
75%      3.500000e+01
max      9.000000e+01
Name: IDADE, dtype: float64

In [8]:
#lista_renomeada = [col.replace('_', ' ').upper() for col in lista_colunas]
lista_renomeada = ['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA','MODALIDADE DE ENSINO','CURSO','TURNO','SEXO','RAÇA','DEFICIENTE FÍSICO','REGIÃO','UF','MUNICÍPIO','IDADE']
df_chars.rename(columns=dict(zip(lista_colunas, lista_renomeada)), inplace=True)
df_chars.columns

Index(['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA',
       'MODALIDADE DE ENSINO', 'CURSO', 'TURNO', 'SEXO', 'RAÇA',
       'DEFICIENTE FÍSICO', 'REGIÃO', 'UF', 'MUNICÍPIO', 'IDADE'],
      dtype='object')

### Criando dropdowns

In [9]:
df_chars["FAIXA DE IDADE"] = pd.cut(
    df_chars["IDADE"],
    bins=[0, 18, 30, 40, 59, float("inf")],
    labels=["ADOLESCENTE (ATÉ 18)", "ADULTO JOVEM (19-30)", "ADULTO (31-40)",'MEIA IDADE (41-59)', "IDOSO (60+)"],
    right=False
)

df_chars['FAIXA DE IDADE'].value_counts()

FAIXA DE IDADE
ADULTO JOVEM (19-30)    1263867
ADULTO (31-40)          1047054
MEIA IDADE (41-59)       356562
IDOSO (60+)               17845
ADOLESCENTE (ATÉ 18)        442
Name: count, dtype: int64

In [10]:
df_chars["FAIXA DE IDADE"] = pd.cut(
    df_chars["IDADE"],
    bins=[0, 18, 30, 40, 59, float("inf")],
    labels=["ADOLESCENTE (ATÉ 18)", "ADULTO JOVEM (19-30)", "ADULTO (31-40)",'MEIA IDADE (41-59)', "IDOSO (60+)"],
    right=False
)

#df_chars['FAIXA DE IDADE'].value_counts()

ordem = ["ADOLESCENTE (ATÉ 18)", "ADULTO JOVEM (19-30)", "ADULTO (31-40)",'MEIA IDADE (41-59)', "IDOSO (60+)"]

faixas = [f for f in ordem if f in df_chars["FAIXA DE IDADE"].unique()]

In [11]:
tipos = ['TODOS OS TIPOS'] + sorted(df_chars['TIPO DE BOLSA'].unique())
anos = ['TODOS OS ANOS']
anos.extend(sorted(df_chars['ANO DE CONCESSÃO'].unique()))
modos = ['QUANTIDADE', 'CRESCIMENTO PERCENTUAL']
regioes = ['PAÍS INTEIRO', 'COMPARAR REGIÕES'] + sorted(df_chars['REGIÃO'].unique())

In [12]:
dropdown_tipo = widgets.Dropdown(
    options=tipos, value=tipos[0],
    description='TIPO DE BOLSA'
)


dropdown_modo = widgets.Dropdown(
    options=modos, value='QUANTIDADE',
    description='ESCALA'
)

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description='ANO'
)

dropdown_faixa = widgets.Dropdown(
    options=['TODAS AS IDADES'] + faixas,
    value='TODAS AS IDADES',
    description='FAIXA DE IDADE'
)

dropdown_regiao = widgets.Dropdown(
    options=regioes,
    value='PAÍS INTEIRO',
    description='REGIÃO'
)

dropdown_ano.style = {'description_width': 'initial'}
dropdown_faixa.style = {'description_width': 'initial'}
dropdown_modo.style = {'description_width': 'initial'}
dropdown_regiao.style = {'description_width': 'initial'}
dropdown_tipo.style = {'description_width': 'initial'}

dropdown_ano.layout = widgets.Layout(width='180px')
dropdown_faixa.layout = widgets.Layout(width='250px')
dropdown_tipo.layout = widgets.Layout(width='320px')
dropdown_regiao.layout = widgets.Layout(width='230px')
dropdown_modo.layout = widgets.Layout(width='280px')

## Número de bolsas por idade, tipo e gênero

In [13]:
def total_idade_tipo_e_sexo(ano, faixa_idade):
    base = df_chars[df_chars['ANO DE CONCESSÃO'] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_chars

    if faixa_idade != 'TODAS AS IDADES':
        base = base[base['FAIXA DE IDADE'].astype(str) == faixa_idade]

    agrupado = (
        base.groupby(['TIPO DE BOLSA', 'SEXO'])
            .size()
            .reset_index(name='QUANTIDADE')
    )

    tabela = agrupado.pivot_table(
        index='TIPO DE BOLSA',
        columns='SEXO',
        values='QUANTIDADE',
        fill_value=0,
    ).reset_index()

    if 'F' not in tabela.columns:
        tabela['F'] = 0
    if 'M' not in tabela.columns:
        tabela['M'] = 0
    
    tabela['TOTAL'] = tabela['F'] + tabela['M']

    tabela = tabela.rename(columns={
        'F': 'FEMININO',
        'M': 'MASCULINO'
    })
    
    if tabela.empty:
        print('Sem dados para esse filtro.')
        return
    
    titulo = f'Quantidade de bolsas por sexo e tipo de bolsa de 2005 a 2019' if ano == 'TODOS OS ANOS' else f'Quantidade de bolsas por sexo e tipo de bolsa em {ano}'   
    
    if faixa_idade != 'TODAS AS IDADES':
        titulo += f' entre {faixa_idade.lower().replace("adulto", "adultos").replace("jovem", "jovens").replace("idoso", "idosos")}'
    
    fig = px.bar(
        tabela,
        x='TIPO DE BOLSA',
        y=['TOTAL', 'FEMININO', 'MASCULINO'],
        barmode='group',
        title=titulo,
        text_auto=True,
        color_discrete_map={
            'TOTAL': 'green',
            'FEMININO': 'magenta',
            'MASCULINO': 'blue'
        }
    )

    fig.update_traces(
    hovertemplate=
        'TIPO: %{x}<br>'
        + 'QTDE: %{y:,.0f}'.replace(',', '.')
        + '<extra></extra>',
        textposition='inside',
        texttemplate='%{y:,.0f}'.replace(',', '.')
    )
    
    fig.update_layout(
        height=600,
        width=1000,
        xaxis_title='TIPO DE BOLSA',
        yaxis_title='QUANTIDADE DE BOLSAS',
        uniformtext_minsize=6,
        uniformtext_mode='hide',
        legend_title='LEGENDA',
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.'
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=total_idade_tipo_e_sexo,
    controls={'ano': dropdown_ano, 'faixa_idade': dropdown_faixa},
    folder=f'../figures/{total_idade_tipo_e_sexo.__name__}',
    default_title='total_idade_tipo_e_sexo'
)

Output()

In [21]:
def total_idade_tipo_e_sexo_animado():

    base = df_chars.copy()

    base['SEXO'] = base['SEXO'].map({'F': 'FEMININO', 'M': 'MASCULINO'})
    base_total = base.copy()
    base_total['SEXO'] = 'TOTAL'

    base = pd.concat([base, base_total], ignore_index=True)

    # Agrupar por ano, faixa de idade e sexo
    agrupado = (
        base
        .groupby(['ANO DE CONCESSÃO', 'FAIXA DE IDADE', 'SEXO'])
        .size()
        .reset_index(name='QUANTIDADE DE BOLSAS')
    )

    #agrupado.head()


    # Gráfico em Plotly com slider de ano e barras agrupadas por sexo
    fig = px.bar(
        agrupado,
        x='FAIXA DE IDADE',
        y='QUANTIDADE DE BOLSAS',
        color='SEXO',
        animation_frame='ANO DE CONCESSÃO',
        barmode='group',
        labels={
            'FAIXA DE IDADE': 'Faixa de idade',
            'QUANTIDADE DE BOLSAS': 'Quantidade de bolsas',
            'SEXO': 'Sexo',
            'ANO DE CONCESSÃO': 'Ano de concessão'
        },
        color_discrete_map={
            'TOTAL': 'green',
            'FEMININO': 'magenta',
            'MASCULINO': 'blue'
        },
        title='Bolsas do Prouni por faixa de idade, ano de concessão e sexo'
    )
    
    max_y = agrupado['QUANTIDADE DE BOLSAS'].max() 

    fig.update_yaxes(range=[0, max_y * 1.1])

    
    fig.update_layout(
        xaxis_tickangle=-45,
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        height=1000,
        uniformtext_minsize=6,
        uniformtext_mode='show',
        legend_title='Sexo',
    )
    fig.show()

total_idade_tipo_e_sexo_animado()

C:\Users\felip\AppData\Local\Temp\ipykernel_30376\1567575110.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [15]:
def crescimento_idade_tipo_e_sexo(faixa_idade, tipo_bolsa, modo):
    base = df_chars.copy()

    # Mapear gênero
    base['SEXO'] = base['SEXO'].map({
        'F': 'FEMININO',
        'M': 'MASCULINO'
    })

    if faixa_idade != 'TODAS AS IDADES':
        base = base[base['FAIXA DE IDADE'].astype(str) == faixa_idade]

    if tipo_bolsa != 'TODOS OS TIPOS':
        base = base[base['TIPO DE BOLSA'] == tipo_bolsa]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

    # agrega por raça e ano
    agrupado = (
        base.groupby(['SEXO', 'ANO DE CONCESSÃO'])
            .size()
            .reset_index(name='QUANTIDADE')
            .sort_values(['SEXO', 'ANO DE CONCESSÃO'])
    )

    # crescimento percentual por raça
    def crescimento_percentual_por_grupo(s):
        primeiro = s.iloc[0]
        if primeiro == 0:
            return pd.Series([pd.NA] * len(s), index=s.index)
        return (s / primeiro - 1) * 100

    agrupado['CRESCIMENTO PERCENTUAL'] = (
        agrupado.groupby('SEXO')['QUANTIDADE'].transform(crescimento_percentual_por_grupo)
    )

    # Define métrica e label
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    else:
        metrica = 'CRESCIMENTO PERCENTUAL'
        y_label = 'Crescimento em relação ao primeiro ano (%)'

    # título
    tipo_titulo = 'todas as bolsas' if tipo_bolsa == 'TODOS OS TIPOS' else tipo_bolsa.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('complementar', 'complementares').replace('integral', 'integrais')
    if faixa_idade == 'TODAS AS IDADES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} comparando sexo'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} na faixa etária {faixa_idade} anos'

    fig = px.line(
        agrupado,
        x='ANO DE CONCESSÃO',
        y=metrica,
        color='SEXO',
        markers=True,
        hover_data={
            'ANO DE CONCESSÃO': True,
            'SEXO': True,
            'QUANTIDADE': ':.0f',
            'CRESCIMENTO PERCENTUAL': ':.1f' if modo != 'QUANTIDADE' else False,
        },
        color_discrete_map={
            'FEMININO': 'magenta',
            'MASCULINO': 'blue',
            'TOTAL': 'green'
        },
        title=titulo,
    )

    # ordem da legenda
    ordem = ['FEMININO','MASCULINO']
    fig.data = tuple(sorted(fig.data, key=lambda tr: ordem.index(tr.name) if tr.name in ordem else len(ordem)))

    # destaque do TOTAL (opcional)
    for tr in fig.data:
        if tr.name == 'TOTAL':
            tr.line.dash = 'dash'
            tr.line.width = 4

    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title='Ano de Concessão',
        yaxis_title=y_label,
        legend_title='Sexo',
        xaxis=dict(dtick=1),
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
        width=1200,
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_idade_tipo_e_sexo,
    controls={'faixa_idade': dropdown_faixa,'tipo_bolsa': dropdown_tipo,'modo': dropdown_modo},
    folder=f'../figures/{crescimento_idade_tipo_e_sexo.__name__}',
    default_title='crescimento_idade_tipo_e_sexo'
)

Output()

## Número de bolsas por idade, tipo e raça

In [16]:
pd.DataFrame(df_chars['RAÇA']).value_counts()

RAÇA         
Branca           1160045
Parda            1111553
Preta             340994
Amarela            47077
Nao Informada      23237
Indigena            2864
Name: count, dtype: int64

In [17]:
def total_idade_tipo_e_raça(ano, faixa_idade):
    base = df_chars[df_chars['ANO DE CONCESSÃO'] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_chars
    
    if faixa_idade != 'TODAS AS IDADES':
        base = base[base['FAIXA DE IDADE'].astype(str) == faixa_idade]

    agrupado = (
        base.groupby(['TIPO DE BOLSA', 'RAÇA'])
            .size()
            .reset_index(name='QUANTIDADE')
    )

    tabela = agrupado.pivot_table(
        index='TIPO DE BOLSA',
        columns='RAÇA',
        values='QUANTIDADE',
        fill_value=0,
    ).reset_index()

    if 'Branca' not in tabela.columns:
        tabela['Branca'] = 0
    if 'Preta' not in tabela.columns:
        tabela['Preta'] = 0
    if 'Parda' not in tabela.columns:
        tabela['Parda'] = 0
    if 'Amarela' not in tabela.columns:
        tabela['Amarela'] = 0
    if 'Indigena' not in tabela.columns:
        tabela['Indigena'] = 0
    if 'Nao Informada' not in tabela.columns:
        tabela['Nao Informada'] = 0

    tabela['TOTAL'] = tabela['Branca'] + tabela['Preta'] + tabela['Parda'] + tabela['Amarela'] + tabela['Indigena'] + tabela['Nao Informada']

    tabela = tabela.rename(columns={
        'Branca' : 'BRANCA',
        'Preta' : 'PRETA',
        'Parda' : 'PARDA',
        'Amarela' : 'AMARELA',
        'Indigena' : 'INDÍGENA',
        'Nao Informada' : 'NÃO INFORMADA'
    })
    
    if tabela.empty:
        print('Sem dados para esse filtro.')
        return

    titulo = f'Quantidade de bolsas por tipo e raça de 2005 a 2019' if ano == 'TODOS OS ANOS' else f'Quantidade de bolsas por tipo e raça em {ano}'
    
    fig = px.bar(
        tabela,
        x='TIPO DE BOLSA',
        y=['TOTAL', 'BRANCA', 'PRETA', 'PARDA', 'AMARELA', 'INDÍGENA', 'NÃO INFORMADA'],
        barmode='group',
        title=titulo,
        text_auto=True,
        color_discrete_map={
            'TOTAL': 'green',
            'BRANCA': 'cyan',
            'PRETA': 'black',
            'PARDA': 'blue',
            'AMARELA': 'yellow',
            'INDÍGENA': 'red',
            'NÃO INFORMADA': 'magenta'
        }
    )

    # separadores pt-BR: decimal ',' e milhar '.'
    fig.update_layout(separators=',.')

    # cada trace vira uma 'raça', então a gente replica o nome dela para cada barra
    for tr in fig.data:
        nome = 'TOTAL' if tr.name == 'TOTAL' else tr.name
        tr.customdata = [nome] * len(tr.x)

    fig.update_traces(
        hovertemplate='TIPO: %{x}<br>QTDE: %{y:,.0f}<br>RAÇA: %{customdata}<extra></extra>',
        textposition='outside',
        texttemplate='%{y:,.0f}'
    )

    fig.update_yaxes(
    dtick=200_000
    )

    
    fig.update_layout(
        height=1000,
        width=1400,
        xaxis_title='TIPO DE BOLSA',
        yaxis_title='QUANTIDADE DE BOLSAS',
        uniformtext_minsize=6,
        uniformtext_mode='show',
        legend_title='RAÇA',
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black')
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=total_idade_tipo_e_raça,
    controls={'ano': dropdown_ano, 'faixa_idade': dropdown_faixa},
    folder=f'../figures/{total_idade_tipo_e_raça.__name__}',
    default_title='total_idade_tipo_e_raça'
)

Output()

In [18]:
def crescimento_idade_tipo_e_raca(faixa_idade, tipo_bolsa, modo):
    base = df_chars.copy()

    # padroniza raça
    base['RAÇA'] = base['RAÇA'].map({
        'Branca': 'BRANCA',
        'Preta': 'PRETA',
        'Parda': 'PARDA',
        'Amarela': 'AMARELA',
        'Indigena': 'INDÍGENA',
        'Nao Informada': 'NÃO INFORMADA'
    })

    # filtro faixa
    if faixa_idade != 'TODAS AS IDADES':
        base = base[base['FAIXA DE IDADE'].astype(str) == faixa_idade]

    # filtro tipo
    if tipo_bolsa != 'TODOS OS TIPOS':
        base = base[base['TIPO DE BOLSA'] == tipo_bolsa]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

    # agrega por raça e ano
    agrupado = (
        base.groupby(['RAÇA', 'ANO DE CONCESSÃO'])
            .size()
            .reset_index(name='QUANTIDADE')
            .sort_values(['RAÇA', 'ANO DE CONCESSÃO'])
    )

    # crescimento percentual por raça
    def crescimento_percentual_por_grupo(s):
        primeiro = s.iloc[0]
        if primeiro == 0:
            return pd.Series([pd.NA] * len(s), index=s.index)
        return (s / primeiro - 1) * 100

    agrupado['CRESCIMENTO PERCENTUAL'] = (
        agrupado.groupby('RAÇA')['QUANTIDADE'].transform(crescimento_percentual_por_grupo)
    )

    # Define métrica e label
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    else:
        metrica = 'CRESCIMENTO PERCENTUAL'
        y_label = 'Crescimento em relação ao primeiro ano (%)'

    # título
    tipo_titulo = 'todas as bolsas' if tipo_bolsa == 'TODOS OS TIPOS' else tipo_bolsa.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('complementar', 'complementares').replace('integral', 'integrais')
    if faixa_idade == 'TODAS AS IDADES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} comparando raças'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} na faixa etária {faixa_idade} anos'

    fig = px.line(
        agrupado,
        x='ANO DE CONCESSÃO',
        y=metrica,
        color='RAÇA',
        markers=True,
        hover_data={
            'ANO DE CONCESSÃO': True,
            'RAÇA': True,
            'QUANTIDADE': ':.0f',
            'CRESCIMENTO PERCENTUAL': ':.1f' if modo != 'QUANTIDADE' else False,
        },
        color_discrete_map={
            'BRANCA': 'cyan',
            'PRETA': 'black',
            'PARDA': 'blue',
            'AMARELA': 'yellow',
            'INDÍGENA': 'red',
            'NÃO INFORMADA': 'magenta'
        },
        title=titulo,
    )

    # ordem da legenda
    ordem = ['AMARELA', 'BRANCA','INDÍGENA','PARDA', 'PRETA', 'NÃO INFORMADA']
    fig.data = tuple(sorted(fig.data, key=lambda tr: ordem.index(tr.name) if tr.name in ordem else len(ordem)))

    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title='Ano de Concessão',
        yaxis_title=y_label,
        legend_title='Raça',
        xaxis=dict(dtick=1),
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
        width=1200,
    )

    return fig
    
fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_idade_tipo_e_raca,
    controls={'faixa_idade': dropdown_faixa,'tipo_bolsa': dropdown_tipo,'modo': dropdown_modo},
    folder=f'../figures/{crescimento_idade_tipo_e_raca.__name__}',
    default_title='crescimento_idade_tipo_e_raca'
)

Output()

## Crescimento de cada tipo de bolsa ao longo dos anos no país

In [19]:
def crescimento_bolsas_anos(tipo, regiao, modo):
    # Paleta opcional por tipo
    cores_por_tipo = {
        'TODOS OS TIPOS': 'yellow',
        'BOLSA INTEGRAL': 'yellow',
        'BOLSA PARCIAL 50%': 'yellow',
        'BOLSA COMPLEMENTAR 25%': 'yellow'
    }

    cores_por_regiao = {
        'PAÍS INTEIRO': 'yellow',
        'SUL': 'blue',
        'NORDESTE': 'magenta',
        'NORTE': 'green',
        'CENTRO-OESTE': 'black',
        'SUDESTE': 'red',
    }
        
    base = df_chars.copy()

    # Filtra tipo só se não for 'TODOS OS TIPOS'
    if tipo != 'TODOS OS TIPOS':
        base = base[base['TIPO DE BOLSA'] == tipo]

    if base.empty:
        print('Sem dados para esse filtro.')
        return

    # Define métrica e label
    if modo == 'QUANTIDADE':
        metrica = 'QUANTIDADE'
        y_label = 'Quantidade de bolsas'
    else:
        metrica = 'CRESCIMENTO PERCENTUAL'
        y_label = 'Crescimento em relação ao primeiro ano (%)'

    # Função de crescimento percentual
    def crescimento_percentual(s):
        primeiro = s.iloc[0]
        if primeiro == 0:
            return pd.Series([pd.NA] * len(s), index=s.index)
        return (s / primeiro - 1) * 100

    # Cenário 1: país inteiro
    if regiao == 'PAÍS INTEIRO':
        dados = (
            base.groupby('ANO DE CONCESSÃO')
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values('ANO DE CONCESSÃO')
        )
        dados['TIPO DE BOLSA'] = tipo

        dados['CRESCIMENTO PERCENTUAL'] = crescimento_percentual(dados['QUANTIDADE'])

        cor_tipo = cores_por_tipo.get(tipo, '#1f77b4')

        fig = px.line(
            dados,
            x='ANO DE CONCESSÃO',
            y=metrica,
            markers=True,
            color_discrete_sequence=[cor_tipo],
            hover_data={
                'ANO DE CONCESSÃO': True,
                'TIPO DE BOLSA': True,
                'QUANTIDADE': ':.0f',
                'CRESCIMENTO PERCENTUAL': ':.1f' if modo != 'QUANTIDADE' else False,
            },
        )

    # Cenário 2: uma região específica
    elif regiao in df_chars['REGIÃO'].unique():
        base_reg = base[base['REGIÃO'] == regiao]
        if base_reg.empty:
            print('Sem dados para essa região.')
            return

        dados = (
            base_reg.groupby('ANO DE CONCESSÃO')
                    .size()
                    .reset_index(name='QUANTIDADE')
                    .sort_values('ANO DE CONCESSÃO')
        )
        dados['TIPO DE BOLSA'] = tipo
        dados['REGIÃO'] = regiao

        dados['CRESCIMENTO PERCENTUAL'] = crescimento_percentual(dados['QUANTIDADE'])

        cor_tipo = cores_por_tipo.get(tipo, '#1f77b4')

        fig = px.line(
            dados,
            x='ANO DE CONCESSÃO',
            y=metrica,
            markers=True,
            color_discrete_sequence=[cor_tipo],
            hover_data={
                'ANO DE CONCESSÃO': True,
                'TIPO DE BOLSA': True,
                'REGIÃO': True,
                'QUANTIDADE': ':.0f',
                'CRESCIMENTO PERCENTUAL': ':.1f' if modo != 'QUANTIDADE' else False,
            },
        )

    # Cenário 3: comparar regiões
    elif regiao == 'COMPARAR REGIÕES':
        dados = (
            base.groupby(['ANO DE CONCESSÃO', 'REGIÃO'])
                .size()
                .reset_index(name='QUANTIDADE')
                .sort_values(['REGIÃO', 'ANO DE CONCESSÃO'])
        )

        dados['CRESCIMENTO PERCENTUAL'] = (
            dados.groupby('REGIÃO')['QUANTIDADE'].transform(crescimento_percentual)
        )

        fig = px.line(
            dados,
            x='ANO DE CONCESSÃO',
            y=metrica,
            color='REGIÃO',
            markers=True,
            color_discrete_map=cores_por_regiao,  # aplica tua paleta
            hover_data={
                'ANO DE CONCESSÃO': True,
                'REGIÃO': True,
                'QUANTIDADE': ':.0f',
                'CRESCIMENTO PERCENTUAL': ':.2f' if modo != 'QUANTIDADE' else False,
            },
        )

    else:
        print('Opção de região inválida.')
        return

    tipo_titulo = 'todas as bolsas' if tipo == 'TODOS OS TIPOS' else tipo.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('complementar', 'complementares').replace('integral', 'integrais')
    if regiao == 'COMPARAR REGIÕES':
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} comparando regiões'
    else:
        titulo = f'{modo.lower().capitalize()} de {tipo_titulo} no {regiao.lower()}'

    fig.update_layout(
        xaxis_title='ANO DE CONCESSÃO',
        yaxis_title=y_label,
        xaxis=dict(dtick=1),
        height=650,
        title=titulo,
        plot_bgcolor='darkgray',
        paper_bgcolor='white',
        font=dict(color='black'),
        separators=',.',
    )

    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_bolsas_anos,
    controls={'tipo': dropdown_tipo, 'regiao': dropdown_regiao, 'modo': dropdown_modo},
    folder=f'../figures/{crescimento_bolsas_anos.__name__}',
    default_title='crescimento_bolsas_anos'
)

Output()

## 

## Mapa

In [20]:
# Carrega o geojson local (sem internet)
with open('brazil-states.geojson', encoding='utf-8') as f:
   geojson = json.load(f)

def mapa_bolsas_por_uf(ano, tipo):
   # Filtra por ano
   dados = df_chars[df_chars['ANO DE CONCESSÃO'] == ano].copy()
   
   if ano == 'TODOS OS ANOS':
      dados = df_chars.copy()

   # Filtra por tipo, se não for 'Todos'
   if tipo != 'TODOS OS TIPOS':
      dados = dados[dados['TIPO DE BOLSA'] == tipo]
   
   if dados.empty:
      print('Sem dados para essa combinação de ano e tipo.')
      return
   
   # Agrupa por UF e REGIAO
   mapa_df = (
      dados.groupby(['UF', 'REGIÃO'])
            .size()
            .reset_index(name='TOTAL')
            .sort_values('TOTAL', ascending=False)
   )

   # Título dinâmico
   titulo = f'Bolsas {tipo.lower().replace("bolsa", "bolsas").replace("integral", "integrais").replace('todos os tipos','')} do Prouni em {str(ano).lower().replace("todos os anos", "de 2005 a 2019")}' if tipo != 'TODOS OS ANOS' else f'Bolsas por estado {tipo.lower().replace("bolsa", "bolsas").replace("integral", "integrais").replace('todos os tipos','')} do Prouni de 2005 a 2019'
   # Mapa coroplético por estado
   fig = px.choropleth(
      mapa_df,
      geojson=geojson,
      locations='UF',                  
      featureidkey='properties.sigla', 
      color='TOTAL',
      color_continuous_scale='Turbo',
      hover_data={'UF': True, 'REGIÃO': True, 'TOTAL': True},
      title=titulo,
   )

   fig.update_geos(
      fitbounds='locations',
      visible=False
   )

   fig.update_layout(
      height=500,
      coloraxis_colorbar=dict(title='TOTAL DE BOLSAS'),
      coloraxis_showscale=True,
      margin={'r':0, 't':50, 'l':0, 'b':0},
      width=1000,     
   ),

   return fig
   
fazer_plotly_widget_com_salvar(
    plot_fn=mapa_bolsas_por_uf,
    controls={'ano': dropdown_ano, 'tipo': dropdown_tipo},
    folder=f'../figures/{mapa_bolsas_por_uf.__name__}',
    default_title='mapa_bolsas_por_uf'
)

Output()